In [1]:
# ライブラリのインストール
!pip install -q transformers datasets peft accelerate bitsandbytes huggingface_hub

In [ ]:
# Hugging Face Hub へのログイン
import os

hf_token = input("Enter your HuggingFace token: ")
os.environ["HF_TOKEN"] = hf_token

!huggingface-cli login --token $hf_token --add-to-git-credential
print("Hugging Faceにログインしました。")

In [ ]:
# データセットの読み込みとトークナイズ
from datasets import load_dataset

dataset = load_dataset("json", data_files="data_lora_formatted.json")
print(dataset)

# Mistral-7B-Instructを使用
# プロンプト形式例: "<s>[INST] {instruction} [/INST] {response}</s>"

from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def build_prompt_and_tokenize(example):
    instruction = example["instruction"]
    response = example["response"]
    # プロンプト作成
    prompt_text = f"<s>[INST] {instruction.strip()} [/INST] {response.strip()}</s>"

    # トークナイズ
    tokenized = tokenizer(
        prompt_text,
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    return tokenized

train_dataset = dataset["train"].map(
    build_prompt_and_tokenize,
    remove_columns=dataset["train"].column_names
)

print("トークナイズ後のサンプル:", train_dataset[0])


In [ ]:
# モデルロード、LoRAアダプタ
import torch
from transformers import AutoModelForCausalLM
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,       # 8bit量子化
    device_map="auto",
    trust_remote_code=True,
)

# LoRA対応
model = prepare_model_for_kbit_training(model)

# パディングトークンID
model.config.pad_token_id = tokenizer.pad_token_id

# ハイパーパラメータ
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj","v_proj","k_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

# 学習可能パラメータ
model.print_trainable_parameters()


In [ ]:
# ファインチューニング実行
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# パディング部分を無視して言語モデル学習を行うデータコラレータ
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir="lora-output",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    fp16=True,
    learning_rate=1e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

# キャッシュ無効
model.config.use_cache = False

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()


In [6]:
# 学習済みLoRAアダプタを保存
trainer.model.save_pretrained("lora-output")
tokenizer.save_pretrained("lora-output")

print("学習済みモデル一式を lora-output フォルダに保存しました。")


学習済みモデル一式を lora-output フォルダに保存しました。


In [ ]:
# ベースモデル + LoRAアダプタを再ロードしてgenerate()
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

# 学習済みLoRA
lora_model = PeftModel.from_pretrained(
    base_model,
    "lora-output",  # 自分の出力ディレクトリ
)
lora_model.eval()

# 推論用トークナイザ
tokenizer = AutoTokenizer.from_pretrained("lora-output", trust_remote_code=True)


In [ ]:
prompt = "<s>[INST] アザトース について教えてください。 [/INST]"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = lora_model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))